In [1]:
from bs4 import BeautifulSoup
from openpyxl import load_workbook, Workbook
import random
import requests
import string
import re
import pandas as pd

In [2]:
#“Giełda”, “Linki” i “Filmweb”
wb = Workbook()
wsG = wb.active
wsG.title = "Giełda"
wsL = wb.create_sheet("Linki")
wsF = wb.create_sheet("Filmweb")

print(wb.sheetnames)

['Giełda', 'Linki', 'Filmweb']


In [3]:
# 5 losowych kodów 3-literowych...

codes = []
url = "https://stooq.pl/q/?s="
while len(codes)!=5:
    code=''.join([random.choice(string.ascii_lowercase) for i in range(3)])
    new_url = url+code
    soup = BeautifulSoup(requests.get(new_url).text,"html.parser")
    if "nie istnieje w bazie" not in soup.get_text():
        codes.append(code)
        
codes

['mdp', 'qub', 'gob', 'imc', 'fhd']

In [4]:
# Aktualna cena, procentowa zmiana oraz liczba transakcji...

dane = {"Kod":codes,
        "Kurs":[],
        "Zmiana":[],
        "Transakcje":[]}

for code in codes:
    new_url = url + code
    soup =  BeautifulSoup(requests.get(new_url).text,"html.parser")
    course = float(soup.find(text="Kurs").parent.find("span").get_text())
    changeText = soup.find(text="Zmiana").parent.find(id="aq_"+code+"_m3").get_text()
    change = float(re.sub("[()%]","",changeText))
    transactionTXT = soup.find(text="Transakcje").parent.find("span").get_text().replace(" ","")
    transactions = int(transactionTXT) if transactionTXT else 0

    dane["Kurs"].append(course)
    dane["Zmiana"].append(change)
    dane["Transakcje"].append(transactions)




In [7]:
# Zapisz wyniki...

df=pd.DataFrame(dane)
writer = pd.ExcelWriter(path="H:\Desktop\Siuśta-175ic-B1.xlsx",engine="openpyxl")
book = load_workbook(filename="H:\Desktop\Siuśta-175ic-B1.xlsx")
writer.book=book
df.to_excel(writer, sheet_name="Giełda")
writer.save()

df

,Kod,Kurs,Zmiana,Transakcje
0,mdp,4.04,-5.16,311
1,qub,5.96,0.34,43
2,gob,4.40,-1.79,3
3,imc,15.40,-3.14,30
4,fhd,3.40,-4.49,1


In [8]:
# Znajdź wszystkie linki...

dane = {"Link":[]}
url = "https://www.playgwent.com/pl/"
soup = BeautifulSoup(requests.get(url).text,"html.parser")
links = soup.find_all('a',href=True)
for link in links:
    ref = str(link.get("href"))
    if ref not in dane["Link"]:
        if "http" not in ref:
            ref = url + ref
        dane["Link"].append(ref)

df=pd.DataFrame(dane)
writer = pd.ExcelWriter(path="H:\Desktop\Siuśta-175ic-B1.xlsx",engine="openpyxl")
book = load_workbook(filename="H:\Desktop\Siuśta-175ic-B1.xlsx")
writer.book=book
df.to_excel(writer, sheet_name="Linki")
writer.save()

df

,Link
0,https://www.playgwent.com/pl//pl/play-mobile
1,https://apps.apple.com/us/app/gwent-the-witche...
2,https://www.playgwent.com/pl//pl/play-mobile
3,https://www.playgwent.com/pl//pl/play-mobile
4,https://play.google.com/store/apps/details?id=...
...,...
129,https://www.playgwent.com/pl//pl/contact-support
130,https://eu.store.cdprojektred.com/
131,http://press.cdprojektred.com/
132,https://cdprojektred.com


In [9]:
dane = {"Reżyser":[],
        "Data premiery":[],
        "Boxoffice":[],
        "Ocena filmu":[]}

url = "https://www.filmweb.pl/film/Siedmiu+samurajów-1954-31369"

soup = BeautifulSoup(requests.get(url).text, "html.parser")
    
dane["Reżyser"] = [soup.find(itemprop="director").text.strip()]
dane["Data premiery"] = [soup.find("span", {"class": "block"}).text.strip()]
dane["Boxoffice"] = [soup.find("div", {"class": "filmRating filmRating--hasPanel"}).attrs.get("data-count")]
dane["Ocena filmu"] = [soup.find("div", {"class": "filmRating filmRating--hasPanel"}).attrs.get("data-rate")]

df=pd.DataFrame(dane)
writer = pd.ExcelWriter(path="H:\Desktop\Siuśta-175ic-B1.xlsx",engine="openpyxl")
book = load_workbook(filename="H:\Desktop\Siuśta-175ic-B1.xlsx")
writer.book=book
df.to_excel(writer, sheet_name="Filmweb")
writer.save()

df

,Reżyser,Data premiery,Boxoffice,Ocena filmu
0,Akira Kurosawa,26 kwietnia 1954 (świat),71028,8.050543785095215
